In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

In [2]:
SEED = 4096
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    
np.random.seed(SEED)

In [3]:
file_path = 'iris.data'
df = pd.read_csv(
    file_path,
    header=None,
    names=['SLength', 'SWidth', 'PLength', 'PWidth', 'class'],
)
df.head()

,SLength,SWidth,PLength,PWidth,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
df['class'].astype('category')

0         Iris-setosa
1         Iris-setosa
2         Iris-setosa
3         Iris-setosa
4         Iris-setosa
            ...      
145    Iris-virginica
146    Iris-virginica
147    Iris-virginica
148    Iris-virginica
149    Iris-virginica
Name: class, Length: 150, dtype: category
Categories (3, object): [Iris-setosa, Iris-versicolor, Iris-virginica]

In [5]:
df['class'] = df['class'].astype('category')
df['class'] = df['class'].cat.codes
df.head()

,SLength,SWidth,PLength,PWidth,class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [6]:
n = len(df.index)
print(n)
shuffled_indices = np.random.permutation(n)
df = df.iloc[shuffled_indices]
df.head()

150


,SLength,SWidth,PLength,PWidth,class
65,6.7,3.1,4.4,1.4,1
80,5.5,2.4,3.8,1.1,1
12,4.8,3.0,1.4,0.1,0
131,7.9,3.8,6.4,2.0,2
6,4.6,3.4,1.4,0.3,0


In [7]:
x = df.iloc[:, :4].values.astype(np.float32)
y = df.iloc[:, -1].values.astype(np.int64)

mu = x.mean(axis=0)
span = x.max(axis=0) - x.min(axis=0)

def rescale(inputs):
    return (inputs - mu) / span

x = rescale(x)
print(x[:5])

[[ 0.23796295  0.01916657  0.10870052  0.08388887]
 [-0.09537034 -0.2725      0.00700559 -0.04111111]
 [-0.28981474 -0.02250006 -0.39977404 -0.45777777]
 [ 0.57129633  0.31083325  0.44768357  0.33388886]
 [-0.34537038  0.14416665 -0.39977404 -0.37444443]]


In [8]:
num_train = int(n * .6)
num_test = n - num_train

x_train = x[:num_train]
y_train = y[:num_train]
x_test = x[-num_test:]
y_test = y[-num_test:]

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(90, 4) (90,)
(60, 4) (60,)


In [9]:
class NpDataset(Dataset):
    def __init__(self, data, label):
        assert len(data) == len(label)
        self.data = torch.from_numpy(data)
        self.label = torch.from_numpy(label).long()
        
    def __getitem__(self, index):
        return self.data[index], self.label[index]
    
    def __len__(self):
        return len(self.label)

In [10]:
train_dataset = NpDataset(x_train, y_train)
test_dataset = NpDataset(x_test, y_test)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=False
)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=128,
    shuffle=False
)

len(train_dataloader.dataset)

90

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [12]:
class IrisNN(nn.Module):
    def __init__(self):
        super(IrisNN, self).__init__()
        
        self.fn1 = nn.Linear(4, 6)
        self.fn2 = nn.Linear(6, 3)
        
    def forward(self, x):
        x = F.relu(self.fn1(x))
        x = self.fn2(x)
        return x
    
model = IrisNN()
model.to(device)

IrisNN(
  (fn1): Linear(in_features=4, out_features=6, bias=True)
  (fn2): Linear(in_features=6, out_features=3, bias=True)
)

In [14]:
x, y = next(iter(train_dataloader))
x = x[:5].to(device)
score = model(x)
print(score)

tensor([[ 0.1867, -0.2556, -0.0844],
        [ 0.2047, -0.2388, -0.1191],
        [ 0.0942, -0.3886, -0.2168],
        [ 0.2144, -0.2158, -0.0410],
        [ 0.1083, -0.3722, -0.2131]], device='cuda:0', grad_fn=<AddmmBackward>)


In [15]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.01)

In [16]:
def train():
    model.train()
    
    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)
        n = x.size(0)
        
        optimizer.zero_grad()
        score = model(x)
        loss = loss_fn(score, y)
        
        loss.backward()
        optimizer.step()
        
        predictions = score.max(1, keepdim=True)[1]
        num_correct = predictions.eq(y.view_as(predictions)).sum().item()
        
    acc = num_correct / n
    return loss, acc

In [17]:
def evaluate():
    model.eval()
    
    with torch.no_grad():
        for x, y in test_dataloader:
            x = x.to(device)
            y = y.to(device)
            n = x.size(0)
            score = model(x)
            loss = loss_fn(score, y)
            predictions = score.max(1, keepdim=True)[1]
            num_correct = predictions.eq(y.view_as(predictions)).sum().item()
        
    acc = num_correct / n
    return loss, acc

In [18]:
max_epochs = 200
for epoch in range(max_epochs):
    tr_loss, tr_acc = train()
    eva_loss, eva_acc = evaluate()
    
    print(f'[{epoch}/{max_epochs}] Train loss:{tr_loss:.4f} acc:{tr_acc*100:.2f}% - Test loss:{eva_loss:.4f} acc:{eva_acc*100:.2f}%')

[0/200] Train loss:1.1165 acc:33.33% - Test loss:1.1054 acc:33.33%
[1/200] Train loss:1.1091 acc:33.33% - Test loss:1.0983 acc:33.33%
[2/200] Train loss:1.1021 acc:33.33% - Test loss:1.0913 acc:33.33%
[3/200] Train loss:1.0952 acc:33.33% - Test loss:1.0845 acc:33.33%
[4/200] Train loss:1.0884 acc:33.33% - Test loss:1.0776 acc:33.33%
[5/200] Train loss:1.0815 acc:33.33% - Test loss:1.0706 acc:33.33%
[6/200] Train loss:1.0743 acc:33.33% - Test loss:1.0634 acc:33.33%
[7/200] Train loss:1.0670 acc:33.33% - Test loss:1.0561 acc:33.33%
[8/200] Train loss:1.0594 acc:33.33% - Test loss:1.0487 acc:33.33%
[9/200] Train loss:1.0517 acc:33.33% - Test loss:1.0411 acc:33.33%
[10/200] Train loss:1.0437 acc:33.33% - Test loss:1.0333 acc:51.67%
[11/200] Train loss:1.0356 acc:50.00% - Test loss:1.0254 acc:58.33%
[12/200] Train loss:1.0272 acc:55.56% - Test loss:1.0172 acc:60.00%
[13/200] Train loss:1.0186 acc:56.67% - Test loss:1.0089 acc:63.33%
[14/200] Train loss:1.0097 acc:58.89% - Test loss:1.0004 a

In [19]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[ 1.9710e-06, -6.9410e-06,  3.6700e-06, -2.7989e-05],
        [ 1.3261e-01,  5.9204e-02, -1.0714e+00, -1.3477e+00],
        [-1.2019e-01,  3.9386e-01, -1.0725e+00, -1.1610e+00],
        [-4.9643e-01,  1.0681e+00, -1.0461e+00, -8.9905e-01],
        [ 4.7358e-01, -9.1573e-01,  1.2762e+00,  1.3720e+00],
        [ 1.5159e-01,  6.3947e-02, -1.1793e+00, -1.4325e+00]], device='cuda:0',
       requires_grad=True)
Parameter containing:
tensor([-3.9518e-04,  8.3463e-01,  7.9371e-01,  2.3454e-01,  1.3838e+00,
         8.9896e-01], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 8.4609e-06,  8.3133e-01,  8.0650e-01,  1.3783e+00, -1.0463e+00,
          9.4610e-01],
        [ 7.6175e-06,  7.5574e-01,  3.2095e-01, -1.0136e+00,  6.0803e-01,
          8.7848e-01],
        [ 1.1457e-05, -1.5385e+00, -1.2004e+00, -7.1789e-01,  1.5532e+00,
         -1.5483e+00]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([-0.6264,  0.1544,  0.7091]